In [262]:
import asyncio
from mpyc.runtime import mpc
from mpyc import asyncoro
import mpyc
import mpyc.gmpy
import numpy as np

In [252]:
class SecureOLS:

    def __init__(self, y, X):
        self.secfloat = mpc.SecFxp(256)
        if type(y) is np.ndarray:
            y = self.secfloat.array(y)
        if type(X) is np.ndarray:
            X = self.secfloat.array(X)
        self.y = y
        self.X = X

    def fit(self):
        XXT = self.X.T @ self.X
        XTy = self.X.T @ self.y
        max_XXT = mpc.np_amax(XXT)
        return self.np_inv(XXT/max_XXT) @ XTy / max_XXT

    def np_inv(self, A, niter=100):
        n = A.shape[0]
        X_k = self.secfloat.array(np.eye(n)) / self.norm_1(A)
        for k in range(niter):
            X_k_next = 2 * X_k - X_k @ A @ X_k     
            X_k = X_k_next    
        return X_k_next
        
    def norm_1(self, A):
        n = mpc.np_sgn(A) * A
        n = mpc.np_sum(n, axis=0)
        n = mpc.np_amax(n)
        return n

In [253]:
X = np.array([[1.1, 2.453, 2.4], [3.4, 0.56, 33.2]])
y = np.array([4.5, 1.2])
model = SecureOLS(y, X)

In [254]:
await mpc.output(model.fit())

array([ 0.57129163,  1.62703281, -0.04980512])

In [255]:
import statsmodels.api as sm

sm.OLS(y, X).fit().params

array([ 0.57129163,  1.62703281, -0.04980512])

In [256]:
import pandas as pd
iris = sm.datasets.get_rdataset("iris")
y = iris.data['Species']
X = iris.data.drop(columns=['Species'])
X = sm.add_constant(X)
X = X.to_numpy(dtype=np.float64)
y = pd.get_dummies(y, drop_first=True)["virginica"].to_numpy(dtype=np.float64)

In [257]:
await mpc.output(SecureOLS(y, X).fit())

array([-0.69528186, -0.04587608,  0.20276839,  0.00398791,  0.55177932])

In [258]:
sm.OLS(y, X).fit().params

array([-0.69528186, -0.04587608,  0.20276839,  0.00398791,  0.55177932])

In [259]:
affairs = sm.datasets.get_rdataset('Affairs', 'AER')
y = affairs.data['affairs'].to_numpy(dtype=np.float64)
X = affairs.data.drop(columns=['affairs'])
X = sm.add_constant(X)
X = pd.get_dummies(X, drop_first=True).to_numpy(dtype=np.float64)

In [260]:
await mpc.output(SecureOLS(y, X).fit())

array([ 5.87201014, -0.05097628,  0.16947232, -0.47761363, -0.01374903,
        0.10491597, -0.71187692,  0.05408587, -0.14262446])

In [261]:
sm.OLS(y, X).fit().params

array([ 5.87201014, -0.05097628,  0.16947232, -0.47761363, -0.01374903,
        0.10491597, -0.71187692,  0.05408587, -0.14262446])